In [10]:
import numpy as np
import pandas as pd
from binancepy import MarketData, Client

###TRADING RULES
QUANTPRE = {  'BTCUSDT': 3, 'ETHUSDT': 3, 'BCHUSDT': 2, 'XRPUSDT': 1, 'EOSUSDT': 1, 'LTCUSDT': 3, \
                'TRXUSDT': 0, 'ETCUSDT': 2, 'LINKUSDT': 2, 'XLMUSDT': 0, 'ADAUSDT': 0, 'XMRUSDT': 3, \
                'DASHUSDT': 3, 'ZECUSDT': 3, 'XTZUSDT': 1, 'BNBUSDT': 2, 'ATOMUSDT': 2, 'ONTUSDT': 1, \
                'IOTAUSDT': 1, 'BATUSDT': 1, 'VETUSDT': 0, 'NEOUSDT': 2, 'QTUMUSDT': 1, 'IOSTUSDT': 0 }
PRICEPRE = {  'BTCUSDT': 2, 'ETHUSDT': 2, 'BCHUSDT': 2, 'XRPUSDT': 4, 'EOSUSDT': 3, 'LTCUSDT': 2, \
              'TRXUSDT': 5, 'ETCUSDT':3, 'LINKUSDT': 3  , 'XLMUSDT': 5, 'ADAUSDT': 5, 'XMRUSDT': 2, \
              'DASHUSDT': 2, 'ZECUSDT': 2, 'XTZUSDT': 3, 'BNBUSDT': 3, 'ATOMUSDT': 3, 'ONTUSDT': 4, \
              'IOTAUSDT': 4, 'BATUSDT': 4, 'VETUSDT': 6, 'NEOUSDT': 3, 'QTUMUSDT': 3, 'IOSTUSDT': 6 }

SIDE = {'BUY': 1.0, 'SELL': -1.0}

testnet = False
if testnet:
    # Testnet
    apikey = 'd782f0d42dcf016c32fd56a96d01bb0e120f78802d9de378cdb9fde2b6c841e8' ### INSERT your api key ###
    scrkey = '1ec873bf3137868a9e12e57e8e19cfa5a3a38cec403b741d2f7cd881c51bfd72' ### INSERT your api secret ###
else:
    # Binance
    apikey = 'gXE34TB2lX894BMjcz0D1HkcxdqMHvQLzI3uLwvaBnHwN9zLCCgxLiwPeqTuKpgh'
    scrkey = 'nqwiLqCWepeJyTVq01PNGKR1Ma9niC3GgcyTM8aGrlztRLmkCaP0ixNiqJdAkaxq'

# Problem 1 : Safe Limit Price

In [11]:
def get_possible_price(mk_data, side):
    '''
    Return a safe limit price available on the market
    '''
    mk_depth = mk_data.order_book(limit=5)
    bids = list(float(x[0]) for x in mk_depth['bids'])
    asks = list(float(x[0]) for x in mk_depth['asks'])
    try:
        lim = (side=='BUY')*(bids[0]+bids[1])/2 + (side=='SELL')*(asks[0]+asks[1])/2
        lim = round(lim, PRICEPRE[mk_data.symbol.upper()])
    except:
        print('Not enough bids and asks on the market!')
        lim = None
    return bids, asks, lim

In [27]:
### TEST get_possible_price() function
# symbol = 'BTCUSD Quarterly 0925'
symbol = 'BTCUSDT'
data = MarketData(testnet=testnet, symbol=symbol)
side = 'SELL'
bids, asks, price = get_possible_price(data, side)
if price is not None:
    print('side = ', side)
    if side=='BUY':
        print('best bid = {} \n2nd best bid = {} \nlimit price = {}'.format(bids[0], bids[1], price))
    elif side=='SELL':
        print('best ask = {} \n2nd best ask = {} \nlimit price = {}'.format(asks[0], asks[1], price))

side =  SELL
best ask = 9076.73 
2nd best ask = 9076.87 
limit price = 9076.8


# Problem 2 : Portfolio class

In [19]:
### Portfolio Class:
class Portfolio:
    def __init__( self,
                  client,
                  tradeIns = []):
        '''
        Portfolio class
        '''
        self.client = client
        self.tradeIns = tradeIns.copy()
        self.orderSize = 0
        self.equityDist = {'BUY': 0, 'SELL': 0}
        self.locks = { 'BUY': [], 'SELL': []}
        
    def equity_distribution(self, longPct=0.5, shortPct=0.5, currency='USDT', orderPct=0.1):
        '''
        Retrun number of buy/sell orders with currenty equity
        
            longPct : percentage of equity assigned for buying
        
            shortPct : percentage of equity assigned for selling
        
            orderPct : percentage of equity for a single order
        '''
        balance = self.client.balance()
        equity, available = 0, 0
        for b in balance:
            if b['asset']==currency:
                equity, available = float(b['balance']), float(b['withdrawAvailable'])
                break
        long_equity = longPct*equity
        short_equity = shortPct*equity
        
        info = pd.DataFrame(self.client.position_info())
        if info.shape[0] > 0: 
            short_info = info[info['positionAmt'].astype(float) < 0]
            long_info = info[info['positionAmt'].astype(float) > 0]
            short_position = abs(short_info['positionAmt'].astype(float) @ short_info['entryPrice'].astype(float))
            long_position = abs(long_info['positionAmt'].astype(float) @ long_info['entryPrice'].astype(float))
        else:
            short_position = 0
            long_position = 0
        
        self.orderSize = round(orderPct*equity, 2)
        long_order = int((long_equity - long_position)/self.orderSize)
        short_order = int((short_equity - short_position)/self.orderSize)
        self.equityDist = {'BUY': long_order, 'SELL': short_order}
        return self.orderSize, long_order, short_order
        
    def position_locks(self, prelocks={ 'BUY': [], 'SELL': []}):
        '''
        Check for open positions and return a tradable instruments
        '''
        info = self.client.position_info()
        self.locks = prelocks
        for pos in info:
            amt = float(pos['positionAmt'])
            if amt < 0 and not pos['symbol'] in self.locks['SELL']: self.locks['SELL'].append(pos['symbol'])
            elif amt > 0 and not pos['symbol'] in self.locks['BUY']: self.locks['BUY'].append(pos['symbol'])
        drop_out = set(self.locks['SELL']).intersection(self.locks['BUY'])
        for s in drop_out: self.tradeIns.remove(s)
        return self.tradeIns

In [20]:
### TEST Portfolio class
insIds = [ 'BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'LINKUSDT', 'XTZUSDT', 'LTCUSDT','DASHUSDT', 'EOSUSDT', 'ETCUSDT', 'ONTUSDT' ]
client = Client(apikey, scrkey, testnet=testnet)
portfolio = Portfolio(client, tradeIns=insIds)
order_size, long, short = portfolio.equity_distribution(longPct=0.5, shortPct=0.5, currency='USDT', orderPct=0.1)
tradeIns = portfolio.position_locks()

print('\nInstruments list : \n', tradeIns, '\n')
print('Order Size : {} '.format(order_size))
print('\nNumber of order : \t#long : {} \t#short : {}  '.format(long, short))


Instruments list : 
 ['BTCUSDT', 'ETHUSDT', 'BCHUSDT', 'LINKUSDT', 'XTZUSDT', 'LTCUSDT', 'DASHUSDT', 'EOSUSDT', 'ETCUSDT', 'ONTUSDT'] 

Order Size : 15.01 

Number of order : 	#long : 4 	#short : 4  


# Problem 3: Request limit

In [21]:
from datetime import datetime

def klns_to_df(market_data, feats):
    '''
    Add columns' name to dataFrame received from market and select the columns in feats
    
    feats -> ['_o', '_h', '_l', '_c', '_v']
    '''
    fts = list(str(f) for f in feats)
    _df = pd.DataFrame(market_data, columns = ['_t', '_o', '_h', '_l', '_c', '_v', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore'])
    _df[['_o', '_h', '_l', '_c', '_v']] = _df[['_o', '_h', '_l', '_c', '_v']].astype(float)
    return _df[fts]

def candle_no_limit(mkData, interval, startTime, endTime):
    '''
    Return dataFrame of mkData from startTime to endTime
    '''
    min_in_candle ={'1m':1, '3m':3, '5m':5, '10m':10, '15m':15, '30m': 30, \
                    '1h':60, '4h':240, '6h':360, '12h':720, '1d':1440}
    
    if not isinstance(startTime, int): 
        startTime = int(pd.Timestamp(datetime.strptime(startTime, '%d %b %Y %H:%M:%S')).value/10**6)
    if not isinstance(endTime, int): 
        endTime = int(pd.Timestamp(datetime.strptime(endTime, '%d %b %Y %H:%M:%S')).value/10**6)    
    sub_period = int((endTime - startTime)/(min_in_candle[interval]*60*1000))
    sub_start = startTime
    kln_df = None
    while sub_period > 0:
        numklns = min(sub_period, 500)
        market_df = data.candles_data(interval=interval, startTime=sub_start, limit=numklns)
        sub_df = klns_to_df(market_df, ['_t', '_o', '_h', '_l', '_c', '_v'])
        if kln_df is None:
            kln_df = pd.DataFrame(sub_df)
        else:
            kln_df = kln_df.append(pd.DataFrame(sub_df), ignore_index=True)
        sub_period -= numklns
        sub_start += numklns*min_in_candle[interval]*60*1000
    return kln_df

In [22]:
### Test candle_no_limit() function
symbol = 'BTCUSDT'
data = MarketData(testnet=testnet, symbol=symbol)
t_start = '01 Jun 2020 00:00:00' 
t_end = '03 Jun 2020 00:00:00'
klns = candle_no_limit(data, interval='3m', startTime=t_start, endTime=t_end)
klns.head()

,_t,_o,_h,_l,_c,_v
0,1590969600000,9446.49,9447.64,9428.40,9439.99,1045.443
1,1590969780000,9439.99,9448.97,9420.85,9436.93,801.558
2,1590969960000,9436.94,9471.42,9436.45,9465.00,1171.385
3,1590970140000,9465.43,9488.85,9463.46,9475.51,984.391
4,1590970320000,9475.51,9490.00,9472.90,9481.59,736.379


# James Added test for each of the functions

In [24]:
# call rest api directly in brower
https://api.binance.com/api/v1/ticker/price?symbol=BTCUSDT

SyntaxError: invalid syntax (<ipython-input-24-91c6f0ac07f6>, line 2)

# ping and other test - James

In [29]:
# test ping functionhttp: 
data.server_time()

{'serverTime': 1594908536076}

In [30]:
data.exchange_info()

{'timezone': 'UTC',
 'serverTime': 1594908572216,
 'rateLimits': [{'rateLimitType': 'REQUEST_WEIGHT',
   'interval': 'MINUTE',
   'intervalNum': 1,
   'limit': 2400},
  {'rateLimitType': 'ORDERS',
   'interval': 'MINUTE',
   'intervalNum': 1,
   'limit': 1200}],
 'exchangeFilters': [],
 'symbols': [{'symbol': 'BTCUSDT',
   'status': 'TRADING',
   'maintMarginPercent': '2.5000',
   'requiredMarginPercent': '5.0000',
   'baseAsset': 'BTC',
   'quoteAsset': 'USDT',
   'pricePrecision': 2,
   'quantityPrecision': 3,
   'baseAssetPrecision': 8,
   'quotePrecision': 8,
   'filters': [{'minPrice': '0.01',
     'maxPrice': '100000',
     'filterType': 'PRICE_FILTER',
     'tickSize': '0.01'},
    {'stepSize': '0.001',
     'filterType': 'LOT_SIZE',
     'maxQty': '1000',
     'minQty': '0.001'},
    {'stepSize': '0.001',
     'filterType': 'MARKET_LOT_SIZE',
     'maxQty': '2439.475',
     'minQty': '0.001'},
    {'limit': 0, 'filterType': 'MAX_NUM_ORDERS'},
    {'limit': 0, 'filterType': 'MAX

In [37]:
data.recent_trades()

[{'id': 162917810,
  'price': '9072.54',
  'qty': '0.004',
  'quoteQty': '36.29',
  'time': 1594908901259,
  'isBuyerMaker': False},
 {'id': 162917811,
  'price': '9072.64',
  'qty': '0.185',
  'quoteQty': '1678.43',
  'time': 1594908901259,
  'isBuyerMaker': False},
 {'id': 162917812,
  'price': '9072.69',
  'qty': '1.000',
  'quoteQty': '9072.69',
  'time': 1594908901259,
  'isBuyerMaker': False},
 {'id': 162917813,
  'price': '9072.70',
  'qty': '10.000',
  'quoteQty': '90727.00',
  'time': 1594908901259,
  'isBuyerMaker': False},
 {'id': 162917814,
  'price': '9072.72',
  'qty': '0.052',
  'quoteQty': '471.78',
  'time': 1594908901259,
  'isBuyerMaker': False},
 {'id': 162917815,
  'price': '9072.73',
  'qty': '0.191',
  'quoteQty': '1732.89',
  'time': 1594908901259,
  'isBuyerMaker': False},
 {'id': 162917816,
  'price': '9072.77',
  'qty': '0.360',
  'quoteQty': '3266.19',
  'time': 1594908901259,
  'isBuyerMaker': False},
 {'id': 162917817,
  'price': '9072.91',
  'qty': '0.032

In [39]:
data.aggregate_trades()[1]['p']

'9071.75'